# Perform GSEA using GSEAPY  

Following the potocol defined here: https://gseapy.readthedocs.io/en/latest/gseapy_tutorial.html#use-gsea-command-or-gsea


In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina' # mac
import pandas as pd
import gseapy as gp
import numpy as np
import matplotlib.pyplot as plt

In [23]:
gp.__version__

'0.9.9'

In [7]:
def run_GSEA_gene(celltype,nperm=1000,rseed=1000,
             rnkfile="../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.csv",
             glist='../dat/glists/gsea_final_all.gmt',col=0,
             rank_metrics='odds',**kwargs):
    '''
    col=1 - Z, 0 - logFC
    '''

    gene_exp_alpha = pd.read_csv(rnkfile)[['celltype','gene',rank_metrics]]
            
    #gene_exp_alpha.head()
    rnk = gene_exp_alpha.loc[gene_exp_alpha["celltype"]==celltype].drop(columns='celltype').sort_values(by=rank_metrics)
    if(rank_metrics=='odds'):
        rnk['odds'] =np.log2(rnk['odds'])
        rnk=rnk.reset_index(drop=True)
        df=rnk.drop(columns='gene')        
        np.random.seed(seed=rseed)
        for i in  rnk.index[(rnk['odds']==np.inf).tolist()].tolist():
            rnk.iloc[i,1]= float(df[~df.isin([np.inf])].max(0)*(1+np.random.uniform()/100))

        for i in  rnk.index[(rnk['odds']==-np.inf).tolist()].tolist():
            rnk.iloc[i,1]= float(df[~df.isin([-np.inf])].min(0)*(1+np.random.uniform()/100))
        #rnk['odds']=rnk['odds']/max(abs(rnk['odds']))
        
    print(rnk.shape)
    print(rnk.head(1))
    print(rnk.tail(1))
    
    gs_res_a = gp.prerank(rnk=rnk, # or data='./P53_resampling_data.txt'
                     gene_sets=glist, # enrichr library names or gmt file
                     #set permutation_type to phenotype if samples >=15
                     permutation_num=nperm, # reduce number to speed up test
                     outdir=None,  # do not write output to disk
                     no_plot=True, # Skip plotting
                     #weighted_score_type=1,
                     #ascending=False,
                        seed=rseed,
                     min_size=1,
                     max_size=5000,
                     processes=8,**kwargs)
                     #format='png')
    return(gs_res_a)

## Run on complete list 

In [25]:
gs_res_a=run_GSEA_gene(celltype='alpha',nperm=1000,glist='../dat/glists/gsea_final_all.gmt',weighted_score_type=0)
gs_res_b=run_GSEA_gene(celltype='beta',nperm=1000,glist='../dat/glists/gsea_final_all.gmt',weighted_score_type=0)
gs_res_d=run_GSEA_gene(celltype='delta',nperm=1000,glist='../dat/glists/gsea_final_all.gmt',weighted_score_type=0)
display(gs_res_a.res2d.sort_index())
display(gs_res_b.res2d.sort_index())
display(gs_res_d.res2d.sort_index())
gs_res_b.res2d[['es','nes','pval','fdr']].to_csv('../figures/Fig2/subfigs/GSEA_beta_res_final.csv')
gs_res_a.res2d[['es','nes','pval','fdr']].to_csv('../figures/Fig2/subfigs/GSEA_alpha_res_final.csv')
gs_res_d.res2d[['es','nes','pval','fdr']].to_csv('../figures/Fig2/subfigs/GSEA_delta_res_final.csv')

(18771, 2)
   gene      odds
0  MT1G -5.646229
          gene      odds
18770  OSBPL1A  4.822249


2019-11-25 23:16:55,587 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


(18655, 2)
      gene      odds
0  PLA2G4E -4.440136
         gene      odds
18654  ZNF541  4.099641


2019-11-25 23:17:02,325 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


(16219, 2)
     gene      odds
0  AMOTL1 -4.857858
        gene      odds
16218  ACACA  3.179479


2019-11-25 23:17:09,842 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.848253,24.798510,0.000000,0.000000,645,645,TNS1;SLC51B;ABCA13;WNK2;INPP4B;NHSL2;SLC27A5;H...,TNS1;SLC51B;ABCA13;WNK2;INPP4B;NHSL2;SLC27A5;H...
Alpha2,-0.638513,-41.262169,0.000000,0.000000,4508,4508,NFIB;EML1;ALAS1;ADD1;LARP4B;RCL1;DOT1L;OSBP2;P...,SETD8;TSPAN4;APEX1;OSGEP;TGFBR1;KIF20B;FAM20B;...
Beta sub1_xin,0.382010,1.697955,0.032064,0.060418,13,13,FXYD2;FFAR4;RBP4;PPP1R1A;SCGB2A1;TAGLN2;SCGN;P...,FXYD2;FFAR4;RBP4;PPP1R1A;SCGB2A1;TAGLN2;SCGN
Beta sub2_xin,0.187651,1.191069,0.233397,0.375381,28,28,NPY;TFF3;RBP1;STMN2;SCG2;GNAS;ID1;PAM;SEC11C;R...,NPY;TFF3;RBP1;STMN2;SCG2;GNAS;ID1;PAM;SEC11C;R...
Beta sub3_xin,0.248455,0.902420,0.581532,0.564229,13,9,CHGA;INS;ASB9;CPE;IGFBP7;TIMP1;LAMP1;CHGB;CKB,CHGA;INS;ASB9;CPE;IGFBP7;TIMP1;LAMP1;CHGB;CKB
Beta sub4_xin,-0.178403,-3.950282,0.000000,0.000000,390,381,GDF15;PSMF1;GCG;MMP7;TM4SF4;SERPINA1;PACSIN2;K...,BAX;TIMMDC1;EIF4EBP1;UBE2D3;CISD2;TXNL1;TCEA1;...
Beta1,0.523142,17.407015,0.000000,0.000000,856,837,TNS1;SLC51B;EML1;INPP4B;IFI44;ADD1;AMZ1;ZIC1;A...,TNS1;SLC51B;EML1;INPP4B;IFI44;ADD1;AMZ1;ZIC1;A...
Beta2,-0.143717,-4.148517,0.000000,0.000000,671,667,KIAA1217;PKIB;PALLD;MAML3;LTBP1;NRXN3;TNS3;ROB...,IQCB1;EAF2;MDC1;MCOLN3;SEC23B;SYDE2;ECE2;SCAMP...
Delta1,0.684850,5.206072,0.000000,0.000000,39,39,PDE6C;NCAPG2;CASR;F11;DHRS2;CDHR3;EYS;TMEM72;V...,PDE6C;NCAPG2;CASR;F11;DHRS2;CDHR3;EYS;TMEM72;V...


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.475861,14.013444,0.000000,0.000000,645,638,TECR;RHBDL2;PEX5L;VDR;FADS2;SLC43A3;CPA1;PSMF1...,TECR;RHBDL2;PEX5L;VDR;FADS2;SLC43A3;CPA1;PSMF1...
Alpha2,-0.286792,-18.606516,0.000000,0.000000,4508,4473,TOM1L2;NEDD4L;GRIK4;PGM1;TEX2;TUBGCP4;TECR;PDE...,SUB1;ZNF775;POM121;COG5;KRBA1;ST3GAL2;LMF1;CYH...
Beta sub1_xin,0.308645,1.360827,0.128631,0.167140,13,13,FXYD2;SCGB2A1;FFAR4;RBP4;SCGN;PPP1R1A;TUBA4A;P...,FXYD2;SCGB2A1;FFAR4;RBP4;SCGN;PPP1R1A;TUBA4A;P...
Beta sub2_xin,0.279592,1.778189,0.021978,0.036059,28,27,ID1;TFF3;GNAS;RBP1;ID3;IAPP;STMN2;NPY;RPL6;FOS...,ID1;TFF3;GNAS;RBP1;ID3;IAPP;STMN2;NPY;RPL6;FOS...
Beta sub3_xin,0.276176,1.054995,0.360075,0.425083,13,10,INS;ASB9;DLK1;CHGA;CKB;LAMP1;IGFBP7;TIMP1;CPE;...,INS;ASB9;DLK1;CHGA
Beta sub4_xin,-0.222032,-5.081498,0.000000,0.000000,390,380,PSMF1;RAB7A;KRT8;ATP6V0D1;MMP7;COPZ1;PFN1;DBP;...,HNRNPC;LMNA;DNAJB9;SUCLG1;MKKS;HSPD1;HSPE1;SQS...
Beta1,0.804845,27.136097,0.000000,0.000000,856,856,PNMAL2;TOM1L2;TIGIT;ZNF541;GRIK4;PRSS53;DKFZP7...,PNMAL2;TOM1L2;TIGIT;ZNF541;GRIK4;PRSS53;DKFZP7...
Beta2,-0.492507,-14.598299,0.000000,0.000000,671,671,NEDD4L;EIF2AK4;FAM160A1;SDK1;RRAS2;NCOA7;ROBO2...,CREB5;UGDH;C1GALT1;TM4SF4;GAREM;VRK2;SCARB2;CE...
Delta1,0.518357,3.852208,0.000000,0.000000,39,39,CASR;NCOA7;PDE6C;GJD3;GPM6A;STARD8;RPH3AL;SLC3...,CASR;NCOA7;PDE6C;GJD3;GPM6A;STARD8;RPH3AL;SLC3...


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.404461,11.264613,0.000000,0.000000,645,608,GPER1;EDAR;GPR56;PRMT8;WDR7;PDZRN4;ROBO2;NUP35...,GPER1;EDAR;GPR56;PRMT8;WDR7;PDZRN4;ROBO2;NUP35...
Alpha2,-0.217090,-13.652700,0.000000,0.000000,4508,4410,CARM1;C20orf195;GPR56;ABCG2;ST5;ATXN7;MPHOSPH9...,PANK1;SEPT2;KIAA0922;GINM1;SNRPB;CDKL5;RHOF;CP...
Beta sub1_xin,0.343753,1.487255,0.091618,0.125708,13,13,FXYD2;TAGLN2;SCGN;FFAR4;RBP4;ASCL2;SCGB2A1;PRS...,FXYD2;TAGLN2;SCGN;FFAR4;RBP4;ASCL2;SCGB2A1;PRS...
Beta sub2_xin,0.304323,1.757401,0.022044,0.039071,28,25,NENF;RBP1;GPX3;STMN2;PEMT;IGFBP5;ID1;SEC11C;PA...,NENF;RBP1;GPX3;STMN2;PEMT;IGFBP5;ID1;SEC11C;PA...
Beta sub3_xin,0.313527,1.051073,0.379098,0.433101,13,8,CHGA;ASB9;CHGB;LAMP1;IGFBP7;CKB;CPE;TIMP1,CHGA;ASB9;CHGB;LAMP1;IGFBP7;CKB
Beta sub4_xin,-0.131303,-2.944158,0.000000,0.000000,390,376,IFRD1;HLA-E;KRT8;GDF15;SST;MMP7;LRRC59;CHCHD3;...,TMEM176B;TMEM176A;PSMD8;DNAJB1;GNAI1;CCNL1;STR...
Beta1,0.376637,12.163927,0.000000,0.000000,856,802,NGEF;SPON2;ADAMTSL3;GPR56;MOB2;ZIC1;MAP3K7CL;R...,NGEF;SPON2;ADAMTSL3;GPR56;MOB2;ZIC1;MAP3K7CL;R...
Beta2,-0.141600,-4.123349,0.000000,0.000000,671,662,ROBO2;MAP4;MTHFD1L;KIAA1217;NHSL1;KCNIP4;FOXP1...,DCAF6;MPC2;KIF21A;VEGFA;ALCAM;ACSL3;RABIF;IPP;...
Delta1,0.946017,6.968547,0.000000,0.000000,39,39,PKD2L2;CTXN3;STARD8;VWC2;NCAPG2;GJD3;CASR;RPH3...,PKD2L2;CTXN3;STARD8;VWC2;NCAPG2;GJD3;CASR;RPH3...


In [24]:
for score_type in [0,1,1.4,2]:
    print(score_type)
    gs_res_b=run_GSEA_gene(celltype='beta',nperm=1000,glist='../dat/glists/gsea_final_all.gmt',
                           weighted_score_type=score_type,
                           rnkfile="../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.uniq_gene.csv")
    fdr=gs_res_b.res2d.loc['mutli_anticorrelated'].fdr
    print('fdr for scoretype {0}:{1}'.format(score_type,fdr))


0
(14789, 2)
      gene      odds
0  PLA2G4E -3.910672
        gene     odds
14788  CSPG4  4.12579
fdr for scoretype 0:0.025846065971726402
1
(14789, 2)
      gene      odds
0  PLA2G4E -3.910672
        gene     odds
14788  CSPG4  4.12579
fdr for scoretype 1:0.23377960865087538
1.4
(14789, 2)
      gene      odds
0  PLA2G4E -3.910672
        gene     odds
14788  CSPG4  4.12579
fdr for scoretype 1.4:0.40843507214206437
2
(14789, 2)
      gene      odds
0  PLA2G4E -3.910672
        gene     odds
14788  CSPG4  4.12579
fdr for scoretype 2:0.6075027995520716


In [22]:
for i in range(10):
    print(i)
    gs_res_b=run_GSEA_gene(celltype='beta',nperm=1000,glist='../dat/glists/gsea_final_all.gmt',rseed=i
                      rnkfile="../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.uniq_gene.csv")
    fdr=gs_res_b.res2d.loc['mutli_anticorrelated'].fdr
    print(fdr)
    if(fdr<.1):
        break


0
(14789, 2)
      gene      odds
0  PLA2G4E -3.927591
        gene      odds
14788  CSPG4  4.107433
0.24259166248116523
1
(14789, 2)
      gene      odds
0  PLA2G4E -3.905989
        gene      odds
14788  CSPG4  4.096108
0.26751269035532993
2
(14789, 2)
      gene      odds
0  PLA2G4E -3.915276
        gene      odds
14788  CSPG4  4.107299
0.2725925925925926
3
(14789, 2)
      gene      odds
0  PLA2G4E -3.937362
        gene      odds
14788  CSPG4  4.126628
0.2605445808862787
4
(14789, 2)
      gene      odds
0  PLA2G4E -3.910818
        gene      odds
14788  CSPG4  4.118643
0.2297979797979798
5
(14789, 2)
      gene      odds
0  PLA2G4E -3.926258
        gene      odds
14788  CSPG4  4.110082
0.2542372881355932
6
(14789, 2)
      gene      odds
0  PLA2G4E -3.925607
        gene      odds
14788  CSPG4  4.094509
0.22227899846704138
7
(14789, 2)
      gene    odds
0  PLA2G4E -3.9234
        gene      odds
14788  CSPG4  4.130106
0.24339720352149147
8
(14789, 2)
      gene     odds
0  PLA2

In [13]:
gs_res_a=run_GSEA_gene(celltype='alpha',nperm=1000,glist='../dat/glists/gsea_final_all.gmt',
                      rnkfile="../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.uniq_gene.csv")
gs_res_b=run_GSEA_gene(celltype='beta',nperm=1000,glist='../dat/glists/gsea_final_all.gmt',
                      rnkfile="../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.uniq_gene.csv")
gs_res_d=run_GSEA_gene(celltype='delta',nperm=1000,glist='../dat/glists/gsea_final_all.gmt',
                      rnkfile="../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.uniq_gene.csv")
display(gs_res_a.res2d.sort_index())
display(gs_res_b.res2d.sort_index())
display(gs_res_d.res2d.sort_index())
gs_res_b.res2d[['es','nes','pval','fdr']].to_csv('../figures/Fig2/subfigs/GSEA_beta_res_final.csv')
gs_res_a.res2d[['es','nes','pval','fdr']].to_csv('../figures/Fig2/subfigs/GSEA_alpha_res_final.csv')
gs_res_d.res2d[['es','nes','pval','fdr']].to_csv('../figures/Fig2/subfigs/GSEA_delta_res_final.csv')

(14893, 2)
      gene      odds
0  CXorf65 -5.165166
       gene     odds
14892  AVIL  3.61352
(14789, 2)
      gene      odds
0  PLA2G4E -3.910672
        gene     odds
14788  CSPG4  4.12579
(13826, 2)
    gene      odds
0  ACMSD -4.273141
       gene      odds
13825  GBP1  2.821581


2019-11-25 22:53:15,182 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.810028,4.399981,0.000000,0.000000,645,645,COL18A1;SLC51B;NHSL2;CA3;ABCA13;ALX4;NTNG2;F2;...,COL18A1;SLC51B;NHSL2;CA3;ABCA13;ALX4;NTNG2;F2;...
Alpha2,-0.813115,-4.326329,0.000000,0.000000,4508,4508,ARHGEF10L;GCK;PXN;TSHZ1;BAIAP3;LSP1;HNF4A;RHOH...,ZXDC;USP32;C17orf64;PARD6B;SKIL;KPNA4;COX7A2L;...
Beta sub1_xin,0.389510,1.035862,0.397820,0.449519,13,13,FFAR4;RBP4;PPP1R1A;SCGB2A1;SCGN;PRSS23;TAGLN2;...,FFAR4;RBP4;PPP1R1A;SCGB2A1;SCGN
Beta sub2_xin,-0.226528,-0.700529,0.896445,1.000000,28,28,NPY;TFF3;RBP1;SCG2;GNAS;ID1;PAM;SEC11C;RPS4X;G...,FOS;JUNB;RPL6;MLLT11
Beta sub3_xin,0.336408,0.825530,0.659091,0.774541,13,9,CHGA;INS;ASB9;CPE;IGFBP7;TIMP1;LAMP1;CHGB;CKB,CHGA;INS;ASB9
Beta sub4_xin,-0.402305,-1.915815,0.000000,0.000189,390,381,GCG;MMP7;HIST1H1C;AUP1;LURAP1L;TMEM176A;TMEM17...,TRAPPC4;PNO1;PTGES3;RGS2;ACP1;EIF5;SAT1;PSMC4;...
Beta1,0.568262,3.204053,0.000000,0.000000,856,837,IFI44;SLC51B;AL136376.1;PDE6C;MS4A8;LRRC14B;NW...,IFI44;SLC51B;AL136376.1;PDE6C;MS4A8;LRRC14B;NW...
Beta2,-0.505226,-2.528804,0.000000,0.000000,671,667,MLIP;PXN;TSHZ1;FAM129A;PCDH7;GPM6A;SORBS2;ANK2...,TTLL4;LAMC1;FGD4;TMBIM6;IL6R;SLC29A1;ZNF860;OS...
Delta1,0.765395,2.740762,0.000000,0.000000,39,39,PDE6C;F11;DHRS2;CDHR3;EYS;TMEM72;VWC2;SLC30A8;...,PDE6C;F11;DHRS2;CDHR3;EYS;TMEM72;VWC2;SLC30A8;...


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.643040,2.156670,0.000000,0.000000e+00,645,638,RHBDL2;SLC43A3;CPA1;KLKB1;HSH2D;DYSF;AL136376....,RHBDL2;SLC43A3;CPA1;KLKB1;HSH2D;DYSF;AL136376....
Alpha2,-0.522239,NaN,NaN,1.000000e+00,4508,4473,DNASE1;RFX4;GPSM1;PLEKHG5;ADRA2C;HES2;MICB;AMN...,NCOR1;ESYT2;BDH1;AL360181.1;PAOX;CLUH;ZNF440;H...
Beta sub1_xin,0.375253,0.828935,0.702782,8.352961e-01,13,13,SCGB2A1;FFAR4;RBP4;SCGN;PPP1R1A;PRSS23;ASCL2;N...,SCGB2A1;FFAR4;RBP4;SCGN;PPP1R1A
Beta sub2_xin,0.497181,1.253230,0.189542,1.446757e-01,28,27,ID1;TFF3;RBP1;ID3;IAPP;NPY;FOS;GPX3;RGS16;RPS4...,ID1;TFF3;RBP1;ID3;IAPP;NPY;FOS;GPX3;RGS16;RPS4...
Beta sub3_xin,0.761824,1.599361,0.019461,1.030022e-02,13,10,INS;DLK1;CHGA;ASB9;LAMP1;CKB;IGFBP7;TIMP1;CPE;...,INS;DLK1;CHGA
Beta sub4_xin,-0.327396,-1.650668,0.000000,1.000000e+09,390,380,MMP7;MSS51;KRT18;PSMB2;AUP1;DNAJC8;SYF2;ID2;ZN...,RER1;NOP10;SAT1;EIF1;BTG1;NXT1;PFN1;TSC22D1;PP...
Beta1,0.805415,2.706210,0.000000,0.000000e+00,856,856,TIGIT;ZNF541;PRSS53;DKFZP779J2370;CCR8;RHBDL2;...,TIGIT;ZNF541;PRSS53;DKFZP779J2370;CCR8;RHBDL2;...
Beta2,-0.842766,NaN,NaN,1.000000e+00,671,671,CPNE7;FOSB;DNAJB6;SORBS2;MTPAP;RAD21;PHC2;PWWP...,GAREM;VRK2;FAM47E;SCARB2;TMEM64;CEP120;RUFY3;A...
Delta1,0.699796,1.884131,0.000000,0.000000e+00,39,39,PDE6C;GJD3;STARD8;SLC30A8;AC079354.1;CTXN3;USH...,PDE6C;GJD3;STARD8;SLC30A8;AC079354.1;CTXN3;USH...


In [26]:
%%bash 
jupyter nbconvert --execute  --to notebook --inplace plot_GSEA.ipynb

[NbConvertApp] WARNING | Config option `template_path` not recognized by `NotebookExporter`.
[NbConvertApp] Converting notebook plot_GSEA.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: ir
[NbConvertApp] Writing 39471 bytes to plot_GSEA.ipynb


## Run on subset of glists 

## save data 

In [9]:
gs_res_b.res2d[['es','nes','pval','fdr']].to_csv('../figures/Fig2/subfigs/GSEA_beta_res_final.csv')
gs_res_a.res2d[['es','nes','pval','fdr']].to_csv('../figures/Fig2/subfigs/GSEA_alpha_res_final.csv')
gs_res_d.res2d[['es','nes','pval','fdr']].to_csv('../figures/Fig2/subfigs/GSEA_delta_res_final.csv')

In [13]:
from gseapy.plot import gseaplot, heatmap
terms = gs_res_b.res2d.index
terms

Index(['Alpha1', 'Beta1', 'Delta1', 'Beta sub3_xin', 'Beta sub2_xin',
       'Mawla_Beta_1', 'multi_correlated', 'Mawla_Beta_2', 'Beta sub1_xin',
       'mutli_anticorrelated', 'Alpha2', 'Beta sub4_xin', 'Beta2', 'Delta2'],
      dtype='object', name='Term')

In [16]:
from gseapy.plot import gseaplot, heatmap
terms = gs_res_b.res2d.index

gseaplot(gs_res_b.ranking, term='mutli_anticorrelated', 
         **gs_res_b.results['mutli_anticorrelated'],ofname='mutli_anticorrelated'+'_beta_all.pdf')

